In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
import matplotlib.pyplot as plt
# from openpyxl import Workbook
# from openpyxl.utils.dataframe import dataframe_to_rows

In [1]:
HOME = "/home/maurits"
DYNAMOS_ROOT=f"{HOME}/EnergyEfficient_Scattered-Directive"

In [8]:
def bin_age(age_series):
    bins = [-np.inf, 18, 60, np.inf]
    labels = ["Child", "Adult", "Elderly"]
    return (
        pd.cut(age_series, bins=bins, labels=labels, right=True)
        .astype(str)
        .replace("nan", "Unknown")
    )

def create_features(df):
    # Convert 'Age' to numeric, coercing errors to NaN
    df["Age"] = pd.to_numeric(df["Age"], errors="coerce")
    df["Age"] = bin_age(df["Age"])
    df["Cabin"] = df["Cabin"].str[0].fillna("Unknown")
    # df["Title"] = _extract_title(df["Name"])
    df.drop(columns=["PassengerId", "Name", "Ticket", "Name"], inplace=True)
    # keywords = set(df.columns)
    # print(all_keywords)
    df = pd.get_dummies(
        df, columns=["Sex", "Pclass", "Embarked", "Cabin", "Age"]
    )
    # print(df)
    return df

In [33]:
titanic_df = pd.read_csv('/home/maurits/EnergyEfficient_Scattered-Directive/python/hfl-data/titanic.csv', delimiter=',')
# titanic_df = titanic_df.drop(columns='Name')
processed_df = titanic_df.dropna(subset=["Embarked", "Fare"]).copy()
data = create_features(processed_df)

In [2]:
from fabric.plot_script import plot

In [1]:
plot('/home/maurits/EnergyEfficient_Scattered-Directive/fabric/run_dumps/hfl_test_results_2_clients_5_20251007_162552.json')
plot('/home/maurits/EnergyEfficient_Scattered-Directive/fabric/run_dumps/hfl_test_results_2_clients_10_20251007_163131.json')
# plot('/home/maurits/EnergyEfficient_Scattered-Directive/fabric/run_dumps/hfl_test_results_4_clients_5_20251007_162602.json')
# plot('/home/maurits/EnergyEfficient_Scattered-Directive/fabric/run_dumps/hfl_test_results_8_clients_5_20251007_162948.json')

NameError: name 'plot' is not defined

In [ ]:
NOF_CLIENTS = 3
TRAINING_PATH = '/home/maurits/EnergyEfficient_Scattered-Directive/python/hfl-data/titanic_training.csv'

# server_data = pd.read_csv(SERVER_PATH, delimiter=',', index_col=0) 
# client1_data = pd.read_csv(CLIENTONE_PATH, delimiter=',', index_col=0, nrows=296, skiprows=range(1,2))
# client2_data = pd.read_csv(CLIENTTWO_PATH, delimiter=',', index_col=0, nrows=296, skiprows=range(1,297))
# client3_data = pd.read_csv(CLIENTTHREE_PATH, delimiter=',', index_col=0, nrows=296, skiprows=range(1,593))
# client_datasets = [client1_data, client2_data, client3_data]

# client_datasets = [client1_data, client2_data, client3_data]
client_datasets = []
offset = 1

for i in range(NOF_CLIENTS):
    # Number of rows per client
    num_rows = round(889/NOF_CLIENTS)
    print("offset before read", offset)
    dataset = pd.read_csv(TRAINING_PATH, delimiter=',', index_col=0, nrows=num_rows, skiprows=range(1,(1 + offset)))
    client_datasets.append(pd.read_csv(TRAINING_PATH, delimiter=',', index_col=0, nrows=num_rows, skiprows=range(1,(1 + offset))))
    filename = f"/home/maurits/EnergyEfficient_Scattered-Directive/python/hfl-train/datasets/client{i+1}Data.csv"
    dataset.to_csv(filename)
    
    offset = offset + num_rows
    print("offset after read/increase", offset)

offset before read 1
offset after read/increase 297
offset before read 297
offset after read/increase 593
offset before read 593
offset after read/increase 889


In [7]:
PATH = '/home/maurits/EnergyEfficient_Scattered-Directive/python/hfl-data/recs2015_public_v4.csv'

houses_df = pd.read_csv(PATH)
houses_df.head()

,DOEID,REGIONC,DIVISION,METROMICRO,UATYP10,TYPEHUQ,ZTYPEHUQ,CELLAR,ZCELLAR,BASEFIN,...,ZELAMOUNT,NGXBTU,PERIODNG,ZNGAMOUNT,FOXBTU,PERIODFO,ZFOAMOUNT,LPXBTU,PERIODLP,ZLPAMOUNT
0,10001,4,10,METRO,U,2,0,0,0,-2,...,0,103.32,1,0,137.45,-2,-2,91.33,-2,-2
1,10002,3,7,NONE,R,2,0,0,0,-2,...,1,NaN,-2,-2,137.45,-2,-2,91.33,-2,-2
2,10003,3,6,METRO,U,2,0,1,0,1,...,0,100.14,1,0,137.45,-2,-2,91.33,-2,-2
3,10004,2,4,MICRO,C,2,0,1,0,1,...,0,NaN,-2,-2,137.45,-2,-2,91.33,2,0
4,10005,1,2,METRO,U,2,0,1,0,0,...,0,102.83,1,0,137.45,-2,-2,91.33,-2,-2


In [8]:
print('Number of cells before deletion of energy usage: ',houses_df.shape)

# calculate total relevant BTU consumption and delete energy consumption cells afterwards.
houses_df['REL_TOTALBTU']=houses_df[['BTUELSPH','BTUFOSPH','BTUNGSPH','BTULPSPH', 'BTUELCOL','BTUELWTH','BTUFOWTH','BTUNGWTH',
                                    'BTULPWTH','BTUELAHUCOL','BTUNGPLHEAT','BTUELHTBHEAT','BTUNGHTBHEAT','BTUELHTBPMP']].sum(axis=1)
houses_df=houses_df.drop(columns=['DOEID','NWEIGHT','BRRWT1','BRRWT2','BRRWT3','BRRWT4','BRRWT5','BRRWT6','BRRWT7','BRRWT8','BRRWT9','BRRWT10',
                        'BRRWT11','BRRWT12','BRRWT13','BRRWT14','BRRWT15','BRRWT16','BRRWT17','BRRWT18','BRRWT19','BRRWT20','BRRWT21',
                        'BRRWT22','BRRWT23','BRRWT24','BRRWT25','BRRWT26','BRRWT27','BRRWT28','BRRWT29','BRRWT30','BRRWT31','BRRWT32',
                        'BRRWT33','BRRWT34','BRRWT35','BRRWT36','BRRWT37','BRRWT38','BRRWT39','BRRWT40','BRRWT41','BRRWT42','BRRWT43',
                        'BRRWT44','BRRWT45','BRRWT46','BRRWT47','BRRWT48','BRRWT49','BRRWT50','BRRWT51','BRRWT52','BRRWT53','BRRWT54',
                        'BRRWT55','BRRWT56','BRRWT57','BRRWT58','BRRWT59','BRRWT60','BRRWT61','BRRWT62','BRRWT63','BRRWT64','BRRWT65',
                        'BRRWT66','BRRWT67','BRRWT68','BRRWT69','BRRWT70','BRRWT71','BRRWT72','BRRWT73','BRRWT74','BRRWT75','BRRWT76',
                        'BRRWT77','BRRWT78','BRRWT79','BRRWT80','BRRWT81','BRRWT82','BRRWT83','BRRWT84','BRRWT85','BRRWT86','BRRWT87',
                        'BRRWT88','BRRWT89','BRRWT90','BRRWT91','BRRWT92','BRRWT93','BRRWT94','BRRWT95','BRRWT96',
                        'KWH','KWHSPH','KWHCOL','KWHWTH','KWHRFG','KWHRFG1','KWHRFG2','KWHFRZ','KWHCOK','KWHMICRO','KWHCW','KWHCDR',
                        'KWHDWH','KWHLGT','KWHTVREL','KWHTV1','KWHTV2','KWHAHUHEAT','KWHAHUCOL','KWHEVAPCOL','KWHCFAN','KWHDHUM',
                        'KWHHUM','KWHPLPMP','KWHHTBPMP','KWHHTBHEAT','KWHNEC','BTUEL','BTUELSPH','BTUELCOL','BTUELWTH','BTUELRFG',
                        'BTUELRFG1','BTUELRFG2','BTUELFRZ','BTUELCOK','BTUELMICRO','BTUELCW','BTUELCDR','BTUELDWH','BTUELLGT',
                        'BTUELTVREL','BTUELTV1','BTUELTV2','BTUELAHUHEAT','BTUELAHUCOL','BTUELEVAPCOL','BTUELCFAN','BTUELDHUM',
                        'BTUELHUM','BTUELPLPMP','BTUELHTBPMP','BTUELHTBHEAT','BTUELNEC','DOLLAREL','DOLELSPH','DOLELCOL','DOLELWTH',
                        'DOLELRFG','DOLELRFG1','DOLELRFG2','DOLELFRZ','DOLELCOK','DOLELMICRO','DOLELCW','DOLELCDR','DOLELDWH',
                        'DOLELLGT','DOLELTVREL','DOLELTV1','DOLELTV2','DOLELAHUHEAT','DOLELAHUCOL','DOLELEVAPCOL','DOLELCFAN',
                        'DOLELDHUM','DOLELHUM','DOLELPLPMP','DOLELHTBPMP','DOLELHTBHEAT','DOLELNEC','CUFEETNG','CUFEETNGSPH',
                        'CUFEETNGWTH','CUFEETNGCOK','CUFEETNGCDR','CUFEETNGPLHEAT','CUFEETNGHTBHEAT','CUFEETNGNEC','BTUNG','BTUNGSPH',
                        'BTUNGWTH','BTUNGCOK','BTUNGCDR','BTUNGPLHEAT','BTUNGHTBHEAT','BTUNGNEC','DOLLARNG','DOLNGSPH','DOLNGWTH',
                        'DOLNGCOK','DOLNGCDR','DOLNGPLHEAT','DOLNGHTBHEAT','DOLNGNEC','GALLONLP','GALLONLPSPH','GALLONLPWTH',
                        'GALLONLPCOK','GALLONLPCDR','GALLONLPNEC','BTULP','BTULPSPH','BTULPWTH','BTULPCOK','BTULPCDR','BTULPNEC',
                        'DOLLARLP','DOLLPSPH','DOLLPWTH','DOLLPCOK','DOLLPCDR','DOLLPNEC','GALLONFO','GALLONFOSPH','GALLONFOWTH',
                        'GALLONFONEC','BTUFO','BTUFOSPH','BTUFOWTH','BTUFONEC','DOLLARFO','DOLFOSPH','DOLFOWTH','DOLFONEC','TOTALBTU',
                        'TOTALDOL','TOTALBTUSPH','TOTALDOLSPH','TOTALBTUWTH','TOTALDOLWTH','TOTALBTUCOK','TOTALDOLCOK','TOTALBTUCDR',
                        'TOTALDOLCDR','TOTALBTUPL','TOTALDOLPL','TOTALBTUHTB','TOTALDOLHTB','TOTALBTUNEC','TOTALDOLNEC','WOODAMT',
                        'WOODBTU','PELLETAMT','PELLETBTU','ELXBTU','PERIODEL','NGXBTU','PERIODNG','FOXBTU','PERIODFO','LPXBTU',
                        'PERIODLP'])
print('Number of cells after deletion of unnecessary energy usage columns: ',houses_df.shape)

# deleting Z-columns
houses_df=houses_df.drop(columns=['ZADQINSUL', 'ZAGECDRYER', 'ZAGECENAC', 'ZAGECWASH', 'ZAGEDW', 'ZAGEFRZR', 'ZAGERFRI1', 'ZAGERFRI2','ZAIRCOND', 'ZALTFUELPEV', 'ZAMTMICRO', 'ZAPPOTHER', 'ZATHOME', 'ZATTCCOOL', 'ZATTCHEAT', 'ZATTIC', 
                                  'ZATTICFIN', 'ZBACKUP', 'ZBASECOOL', 'ZBASEFIN', 'ZBASEHEAT', 'ZBEDROOMS', 'ZBLENDER', 'ZCABLESAT', 
                                  'ZCELLAR', 'ZCELLPHONE', 'ZCENACHP', 'ZCOFFEE', 'ZCOLDMA', 'ZCOMBODVR', 'ZCOOKTUSE', 'ZCOOLTYPE', 
                                  'ZCROCKPOT', 'ZCWASHER', 'ZDESKTOP', 'ZDISHWASH', 'ZDNTHEAT', 'ZDOOR1SUM', 'ZDRAFTY', 'ZDRYER', 
                                  'ZDRYRFUEL', 'ZDRYRUSE', 'ZDUALCOOKTFUEL', 'ZDUALOVENFUEL', 'ZDVD', 'ZDWASHUSE', 'ZDWCYCLE', 
                                  'ZEDUCATION', 'ZELPAY', 'ZELPERIPH', 'ZEMPLOYHH', 'ZENERGYASST', 'ZENERGYASST11', 'ZENERGYASST12', 
                                  'ZENERGYASST13', 'ZENERGYASST14', 'ZENERGYASST15', 'ZENERGYASSTOTH', 'ZEQUIPAGE', 'ZEQUIPAUX', 
                                  'ZEQUIPAUXTYPE', 'ZEQUIPM', 'ZEQUIPMUSE', 'ZFOODPROC', 'ZFOPAY', 'ZFUELAUX', 'ZFUELH2O', 
                                  'ZFUELH2O2', 'ZFUELHEAT', 'ZFUELPOOL', 'ZFUELTUB', 'ZGARGCOOL', 'ZGARGHEAT', 'ZH2OHEATAPT', 
                                  'ZHEATHOME', 'ZHHAGE', 'ZHHSEX', 'ZHIGHCEIL', 'ZHOTMA', 'ZHOUSEHOLDER_RACE', 'ZICE', 'ZINTERNET', 
                                  'ZINTSTREAM', 'ZINWIRELESS', 'ZKOWNRENT', 'ZLGTIN4', 'ZLGTINCAN', 'ZLGTINCFL', 'ZLGTINCNTL', 
                                  'ZLGTINLED', 'ZLGTINNUM', 'ZLGTOUTCNTL', 'ZLGTOUTNUM', 'ZLOCRFRI2', 'ZLPGPAY', 'ZMICRO', 'ZMOISTURE', 
                                  'ZMONEYPY', 'ZMONPOOL', 'ZMONTUB', 'ZMORETHAN1H2O', 'ZNCOMBATH', 'ZNGPAY', 'ZNHAFBATH', 'ZNHSLDMEM', 
                                  'ZNOACBROKE', 'ZNOACDAYS', 'ZNOACEL', 'ZNOACHELP', 'ZNOHEATBROKE', 'ZNOHEATBULK', 'ZNOHEATDAYS', 
                                  'ZNOHEATEL', 'ZNOHEATHELP', 'ZNOHEATNG', 'ZNOTMOIST', 'ZNUMADULT', 'ZNUMATTICFAN', 'ZNUMBERAC', 
                                  'ZNUMCFAN', 'ZNUMCHILD', 'ZNUMFLOORFAN', 'ZNUMFREEZ', 'ZNUMFRIG', 'ZNUMLAPTOP', 'ZNUMMEAL', 
                                  'ZNUMSMPHONE', 'ZNUMTABLET', 'ZNUMWHOLEFAN', 'ZOCCUPYYRANGE', 'ZOTHROOMS', 'ZOUTGRILL', 
                                  'ZOUTGRILLFUEL', 'ZOUTLET', 'ZOVEN', 'ZOVENFUEL', 'ZOVENUSE', 'ZPAYHELP', 'ZPLAYSTA', 'ZPOOL', 
                                  'ZPRKGPLC1', 'ZPROTHERM', 'ZPROTHERMAC', 'ZRECBATH', 'ZRICECOOK', 'ZRNSETEMP', 'ZROOFTYPE', 'ZSCALEB', 
                                  'ZSCALEE', 'ZSCALEG', 'ZSDESCENT', 'ZSEPCOOKTUSE', 'ZSEPDVR', 'ZSEPOVENUSE', 'ZSIZEOFGARAGE', 
                                  'ZSIZFREEZ', 'ZSIZRFRI1', 'ZSIZRFRI2', 'ZSTORIES', 'ZSTOVE', 'ZSTOVEFUEL', 'ZSTOVEN', 'ZSTOVENFUEL', 
                                  'ZSTUDIO', 'ZSWAMPCOL', 'ZSWIMPOOL', 'ZTEMPGONE', 'ZTEMPGONEAC', 'ZTEMPHOME', 'ZTEMPHOMEAC', 
                                  'ZTEMPNITE', 'ZTEMPNITEAC', 'ZTHERMAIN', 'ZTHERMAINAC', 'ZTOAST', 'ZTOASTOVN', 'ZTOPFRONT', 
                                  'ZTOTROOMS', 'ZTOTSQFT_EN', 'ZTVAUDIOSYS', 'ZTVCOLOR', 'ZTVONWD1', 'ZTVONWD2', 'ZTVONWE1', 'ZTVONWE2', 
                                  'ZTVSIZE1', 'ZTVSIZE2', 'ZTVTYPE1', 'ZTVTYPE2', 'ZTYPEGLASS', 'ZTYPEHUQ', 'ZTYPERFR1', 'ZTYPERFR2', 
                                  'ZUGASHERE', 'ZUPRTFRZR', 'ZUSECENAC', 'ZUSEMOISTURE', 'ZUSENOTMOIST', 'ZUSEWWAC', 'ZVCR', 'ZWALLTYPE', 
                                  'ZWASHLOAD', 'ZWASHTEMP', 'ZWDPELLET', 'ZWHEATAGE', 'ZWHEATSIZ', 'ZWINDOWS', 'ZWINFRAME', 'ZWOODLOGS', 
                                  'ZWWACAGE', 'ZYEARMADERANGE'])
print('Number of cells after deletion of Z-columns: ',houses_df.shape)

Number of cells before deletion of energy usage:  (5686, 759)
Number of cells after deletion of unnecessary energy usage columns:  (5686, 499)
Number of cells after deletion of Z-columns:  (5686, 288)


In [9]:
print(houses_df.shape)
print(houses_df.columns.values)

(5686, 288)
['REGIONC' 'DIVISION' 'METROMICRO' 'UATYP10' 'TYPEHUQ' 'CELLAR' 'BASEFIN'
 'ATTIC' 'ATTICFIN' 'STORIES' 'PRKGPLC1' 'SIZEOFGARAGE' 'KOWNRENT'
 'YEARMADERANGE' 'OCCUPYYRANGE' 'STUDIO' 'BEDROOMS' 'NCOMBATH' 'NHAFBATH'
 'OTHROOMS' 'TOTROOMS' 'WALLTYPE' 'ROOFTYPE' 'HIGHCEIL' 'DOOR1SUM'
 'WINDOWS' 'TYPEGLASS' 'WINFRAME' 'ADQINSUL' 'DRAFTY' 'UGASHERE'
 'SWIMPOOL' 'MONPOOL' 'POOL' 'FUELPOOL' 'RECBATH' 'MONTUB' 'FUELTUB'
 'OUTLET' 'ALTFUELPEV' 'BACKUP' 'SOLAR' 'NUMFRIG' 'SIZRFRI1' 'TYPERFR1'
 'AGERFRI1' 'ICE' 'SIZRFRI2' 'TYPERFR2' 'AGERFRI2' 'LOCRFRI2' 'NUMFREEZ'
 'UPRTFRZR' 'SIZFREEZ' 'AGEFRZR' 'STOVEN' 'STOVENFUEL' 'DUALCOOKTFUEL'
 'DUALOVENFUEL' 'COOKTUSE' 'OVENUSE' 'STOVE' 'STOVEFUEL' 'SEPCOOKTUSE'
 'OVEN' 'OVENFUEL' 'SEPOVENUSE' 'MICRO' 'AMTMICRO' 'OUTGRILL'
 'OUTGRILLFUEL' 'NUMMEAL' 'TOAST' 'TOASTOVN' 'COFFEE' 'CROCKPOT'
 'FOODPROC' 'RICECOOK' 'BLENDER' 'APPOTHER' 'DISHWASH' 'DWASHUSE'
 'DWCYCLE' 'AGEDW' 'CWASHER' 'TOPFRONT' 'WASHLOAD' 'WASHTEMP' 'RNSETEMP'
 'AGECWASH' 'DRYER'

In [10]:
# Mapping certain features.
replacements = {
    'REGIONC' : {1: 'Northeast', 2: 'Midwest', 3:'South', 4 : 'West'},
    'DIVISION' : {1: 'New England', 2 : 'Middle Atlantic' , 3 : 'East North Central', 4 : 'West North Central', 5 :'South Atlantic' , 
                  6: 'East South Central', 7: 'West South Central', 8: 'Mountain North', 9: 'Mountain South' , 10 : ' Pacific '},
    'TYPEHUQ': {1: 'Mobile home' , 2: 'Single-family detached house', 3 : 'single- family attached house', 
                4 : 'Apartment in a building with 2 to 4 units', 5: 'Apartment in a building with 5 or more units'},
    'CELLAR' : {1 : 'Yes', 0 : 'No', -2 : 'Not Applicable'},
    'BASEFIN': {1: 'Yes', 0 : 'No', -2: 'Not Applicable'},
    'ATTIC': {1 : 'Yes', 0: 'No', -2 : 'Not Applicable'},
    'ATTICFIN' : {1 :'Yes', 0 : 'No', -2: 'Not Applicable' },
    'STORIES' : {10 : 'One story', 20 : 'Two stories', 31 : 'Three stories', 32 : 'Four or more stories', 40: 'Split-level', 
                 -2: 'Not Applicable'},
    'PRKGPLC1' : {1 : 'Yes', 0: 'No', -2 : 'Not Applicable'},
    'SIZEOFGARAGE' : {1 : 'One-car garage', 2 : 'Two-car garage', 3: 'Three-or-more-car garage' , -2 : 'Not Applicable'},
    'KOWNRENT': {1 : 'Owned or being bought by someone in your household', 2 : 'Rented', 3 : 'Occupied without payment of rent'},
    'YEARMADERANGE': {1 : 'Before 1950', 2 : '1950 to 1959', 3 : '1960 to 1969', 4 : '1970 to 1979', 5 :'1980 to 1989', 
                      6 : '1990 to 1999', 7 : '2000 to 2009', 8: '2010 to 2015'},
    'OCCUPYYRANGE' : {1 : 'Before 1950', 2 : '1950 to 1959', 3 : '1960 to 1969', 4 : '1970 to 1979', 5 :'1980 to 1989', 
                      6 : '1990 to 1999', 7 : '2000 to 2009', 8: '2010 to 2015'},
    'STUDIO': {1 :'Yes', 0 : 'No', -2 : 'Not Applicable'},
    'WALLTYPE': {1 : 'Brick', 2: 'Wood', 3: 'Siding', 4 : 'Stucco', 5: 'Shingle (composition)', 6:'Stone', 
                 7 : 'Concrete or concrete block', 9: '0ther'},
    'ROOFTYPE': {1 : 'Ceramic or clay tiles', 2: 'Wood shingles/shakes', 3: 'Metal', 4: 'Slate or synthetic shake', 
                 5 : 'Shingles (composition or asphalt)) ', 7: 'Concrete tiles', 9: 'Other', -2 : 'Not Applicable'},
    'HIGHCEIL' : {1: 'Yes', 0 :'No', -2 : 'Not Applicable'},
    'WINDOWS': {10 : '1 to 2', 20: '3 to 5', 30 : '6 to 9', 41 : '10 to 15', 42 : '16 to 19, 50:20 to 29', 60:' 30 or more'},
    'TYPEGLASS': {1 : 'Single-pane glass', 2: 'Double-pane glass', 3 : 'Triple-pane glass'},
    'WINFRAME': {1 : 'Wood', 2 : 'Metal (aluminum)', 3 : 'Vinyl', 4 : 'Composite', 5 : 'Fiberglass'},
    'ADQINSUL': {1: 'Well insulated', 2 : 'Adequately insulated', 3 :'Poorly insulated', 4 : 'Not insulated'},
    'DRAFTY': {1: 'All the time', 2: 'Most of the time',3 : 'Some of the time', 4: 'Never'},
    'SWIMPOOL' : {1 : 'Yes', 0 : 'No', -2 : 'Not Applicable'},
    'POOL' : {1 : 'Yes', 0 : 'No' , -2 : 'Not Applicable'},
    'FUELPOOL': {1 : 'Natural gas from underground pipes', 2: 'Propane (bottled gas)', 3 :'Fuel oil/kerosene', 5:'Electricity', 
                 8:'Solar', 21 :'Some other fuel', -2 : 'Not Applicable'},
    'EQUIPM': {2 : 'Steam/hot water system with radiators or pipes', 3 :'Central furnace', 4 : 'Heat pump', 
               5 : 'Built-in electric units installed in walls, ceilings, baseboards, or floors', 
               6 : 'Built-in floor/wall pipeless furnace', 7 : 'Built-in room heater burning gas, oil, or kerosene', 
               8 : 'Wood-burning stove (cordwood or pellets)', 9 : 'Fireplace', 10 : 'Portable electric heaters', 
               21 : 'Some other equipment',  -2 : 'Not Applicable'},
    'FUELHEAT': {1 : 'Natural gas from underground pipes', 2 : 'Propane (bottled gas)', 3 :'Fuel oil/kerosene', 5: 'Electricity', 
                 7 : 'Wood (cordwood or pellets)', 21 : 'Some other fuel', -2: 'Not Applicable'},
    'EQUIPAGE': {1 : 'Less than 2 years old', 2 : '2 to 4 years old', 3 :'5 to 9 years old', 41 : '10 to 14 years old',
                 42 : '15 to 19 years old', 5 : '20 years old', -2 : 'Not Applicable'},
    'BASEHEAT': {1 : 'Yes', 0 : 'No', -2: 'Not Applicable'},
    'FUELH2O' : {1 : 'Natural gas from underground pipes', 2 : 'Propane (bottled gas)', 3 :'Fuel oil/kerosene', 
                 5 : 'Electricity', 7: 'Wood (cordwood or pellets) ', 8 : 'Solar', 21: 'Some other fuel'},
    'BASECOOL': {1 : 'Yes', 0 : 'No', -2 : 'Not Applicable'}
    }
             # Remap the values of the dataframe
for i in replacements:
    dict=replacements [i]
    houses_df[i] .replace (dict, inplace=True)
houses_df.head()

,REGIONC,DIVISION,METROMICRO,UATYP10,TYPEHUQ,CELLAR,BASEFIN,ATTIC,ATTICFIN,STORIES,...,GWT,DBT1,DBT99,ZWOODAMT,ZPELLETAMT,ZELAMOUNT,ZNGAMOUNT,ZFOAMOUNT,ZLPAMOUNT,REL_TOTALBTU
0,West,Pacific,METRO,U,Single-family detached house,No,Not Applicable,No,Not Applicable,Two stories,...,62.27,97.9,33.0,-2,-2,0,0,-2,-2,37147.894
1,South,West South Central,NONE,R,Single-family detached house,No,Not Applicable,No,Not Applicable,One story,...,67.36,96.8,31.7,-2,-2,1,-2,-2,-2,26287.327
2,South,East South Central,METRO,U,Single-family detached house,Yes,Yes,No,Not Applicable,One story,...,62.32,92.5,24.9,-2,-2,0,0,-2,-2,66173.848
3,Midwest,West North Central,MICRO,C,Single-family detached house,Yes,Yes,No,Not Applicable,One story,...,56.76,91.3,15.9,-2,-2,0,-2,-2,0,73217.584
4,Northeast,Middle Atlantic,METRO,U,Single-family detached house,Yes,No,Yes,No,Two stories,...,50.04,88.4,15.5,-2,-2,0,0,-2,-2,23860.545


In [11]:
# Getting numerical and categorical columns as preparation for one hot encoding
cols=houses_df.columns
num_cols=list(houses_df._get_numeric_data().columns)
cat_cols=list(set(cols)-set(num_cols))
print(f'num_cols:\{num_cols}\n\ncat_cols:{cat_cols}')

num_cols:\['BEDROOMS', 'NCOMBATH', 'NHAFBATH', 'OTHROOMS', 'TOTROOMS', 'DOOR1SUM', 'UGASHERE', 'MONPOOL', 'RECBATH', 'MONTUB', 'FUELTUB', 'OUTLET', 'ALTFUELPEV', 'BACKUP', 'SOLAR', 'NUMFRIG', 'SIZRFRI1', 'TYPERFR1', 'AGERFRI1', 'ICE', 'SIZRFRI2', 'TYPERFR2', 'AGERFRI2', 'LOCRFRI2', 'NUMFREEZ', 'UPRTFRZR', 'SIZFREEZ', 'AGEFRZR', 'STOVEN', 'STOVENFUEL', 'DUALCOOKTFUEL', 'DUALOVENFUEL', 'COOKTUSE', 'OVENUSE', 'STOVE', 'STOVEFUEL', 'SEPCOOKTUSE', 'OVEN', 'OVENFUEL', 'SEPOVENUSE', 'MICRO', 'AMTMICRO', 'OUTGRILL', 'OUTGRILLFUEL', 'NUMMEAL', 'TOAST', 'TOASTOVN', 'COFFEE', 'CROCKPOT', 'FOODPROC', 'RICECOOK', 'BLENDER', 'APPOTHER', 'DISHWASH', 'DWASHUSE', 'DWCYCLE', 'AGEDW', 'CWASHER', 'TOPFRONT', 'WASHLOAD', 'WASHTEMP', 'RNSETEMP', 'AGECWASH', 'DRYER', 'DRYRFUEL', 'DRYRUSE', 'AGECDRYER', 'TVCOLOR', 'TVSIZE1', 'TVTYPE1', 'TVONWD1', 'TVONWE1', 'TVSIZE2', 'TVTYPE2', 'TVONWD2', 'TVONWE2', 'CABLESAT', 'COMBODVR', 'SEPDVR', 'PLAYSTA', 'DVD', 'VCR', 'INTSTREAM', 'TVAUDIOSYS', 'DESKTOP', 'NUMLAPTOP', 

In [12]:
# one hot encoding of categorical columns
houses_df=pd.get_dummies(data=houses_df, columns=cat_cols)
houses_df.head()

,BEDROOMS,NCOMBATH,NHAFBATH,OTHROOMS,TOTROOMS,DOOR1SUM,UGASHERE,MONPOOL,RECBATH,MONTUB,...,WALLTYPE_Stucco,WALLTYPE_Wood,ROOFTYPE_Ceramic or clay tiles,ROOFTYPE_Concrete tiles,ROOFTYPE_Metal,ROOFTYPE_Not Applicable,ROOFTYPE_Other,ROOFTYPE_Shingles (composition or asphalt)),ROOFTYPE_Slate or synthetic shake,ROOFTYPE_Wood shingles/shakes
0,3,2,1,4,7,2,1,-2,0,-2,...,True,False,True,False,False,False,False,False,False,False
1,2,1,0,2,4,0,1,-2,0,-2,...,False,False,False,False,True,False,False,False,False,False
2,4,2,0,5,9,1,1,-2,0,-2,...,False,True,False,False,False,False,False,True,False,False
3,3,2,0,4,7,4,0,-2,0,-2,...,False,False,False,False,False,False,False,True,False,False
4,3,2,0,3,6,1,1,-2,0,-2,...,False,False,False,False,False,False,False,True,False,False


In [13]:
# Remove n/a columns
houses_df=houses_df.drop(columns=['BASECOOL_Not Applicable', 'BASEHEAT_Not Applicable', 'BASEFIN_Not Applicable', 
                                  'ROOFTYPE_Not Applicable', 'FUELPOOL_Not Applicable', 'SWIMPOOL_Not Applicable', 
                                  'STUDIO_Not Applicable', 'ATTICFIN_Not Applicable', 'POOL_Not Applicable', 
                                  'PRKGPLC1_Not Applicable', 'HIGHCEIL_Not Applicable', 'SIZEOFGARAGE_Not Applicable', 
                                  'ATTIC_Not Applicable', 'CELLAR_Not Applicable', 'STORIES_Not Applicable', 
                                  'EQUIPM_Not Applicable', 'EQUIPAGE_Not Applicable'])

In [14]:
# Remove 'no' columns
houses_df=houses_df.drop(columns=['BASEHEAT_No', 'PRKGPLC1_No', 'BASECOOL_No', 'CELLAR_No', 'ATTICFIN_No', 'HIGHCEIL_No', 
                                  'ATTIC_No', 'POOL_No', 'SWIMPOOL_No', 'BASEFIN_No', 'STUDIO_No'])

In [15]:
# Removing behaviour based features, which are not known a priori, and other features which are hard to obtain.
houses_df=houses_df.drop(columns=['HDD50', 'HDD65', 'CDD65', 'CDD30YR', 'DBT99', 'HDD30YR', 'GNDHDD65', 'GWT', 'SWAMPCOL', 'WSF', 'OA_LAT', 'TEMPHOMEAC'])